In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import *
#from pyspark.sql.window import Window

import getpass

In [2]:
userName = getpass.getuser()
print(userName)
id_user = '1000'

band


In [3]:
spark = SparkSession.\
        builder.\
        config("spark.ui.port","0").\
        master("yarn").\
        enableHiveSupport().\
        config("spark.sql.warehouse.dir", f"/user/{userName}/warehouse").\
        appName("Exercises").\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-01-04 13:50:13,487 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [5]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [6]:
# exercise 1:
path = "/public/crime/csv/Crimes.csv"

In [ ]:
'''schema = StructType().\
        add('ID', IntegerType(), True).\
        add('Case Number', StringType(), True).\
        add('Date', StringType(), True).\
        add('Block', StringType(), True).\
        add('IUCR', StringType(), True).\
        add('Primary Type', StringType(), True).\
        add('Description', StringType(), True).\
        add('Location Description' ,StringType(), True).\
        add('Arrest', BooleanType(), True).\
        add('Domestic', BooleanType(), True).\
        add('Beat', StringType(), True).\
        add('District', StringType(), True).\
        add('Ward', IntegerType(), True).\
        add('Community Area', IntegerType(), True).\
        add('FBI code', StringType(), True).\
        add('X Coordinate', LongType(), True).\
        add('Y Coordinate', LongType(), True).\
        add('Year', StringType(), True).\
        add('Updated On', StringType(), True).\
        add('Latitude', FloatType(), True).\
        add('Longitude', FloatType(), True).\
        add('Location', StringType(), True)
crimes = spark.read.format('csv').schema(schema).option('header', True).option('inferschema', True).load(path)
'''

In [7]:
crimes= spark.read.csv(path, header=True, inferSchema=True)

In [8]:
crimes.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
crimes.show(5)

In [ ]:
# Get monthly count of primary crime type

#crimes.select(split(col("Date"), "/")[1].alias("month"), split(split(col("Date"), "/")[2], ' ').alias("year")).show(20)
tmp_crimes = crimes.withColumn('Date', split(col('Date'), ' ')[0])

In [ ]:
print(type(tmp_crimes))

In [ ]:
# sap' xep' theo month tang dan`, count_primary_type giam? dan`
tmp = tmp_crimes.groupBy('Date').agg(count('Primary Type').alias('count_primary_type')).sort(asc(split(col('Date'), '/')[0]), desc('count_primary_type'))

In [ ]:
tmp.show(5)

In [ ]:
tmp = tmp.withColumn('count_primary_type', col('count_primary_type').cast('STRING'))

In [ ]:
tmp.show(5)

In [ ]:
text_path = "/user/exercise/solutions/solution1/crimes_by_type_by_month"

In [ ]:
head = tmp.select(concat_ws(',', col('Date'), col('count_primary_type')).alias(f'{tmp.columns[0]},{tmp.columns[1]}'))

In [ ]:
head.show(5)

In [ ]:
head.write.text(path=text_path, compression='gzip')

In [ ]:
#-------------------------------------------ex2----------------------------------#

In [17]:
def extract_txt(path, headers, symbol=','):
    text_data = spark.sparkContext.textFile(path)
    context = text_data.map(lambda k: k.split(symbol))
    return context.toDF(headers)

In [11]:
path = "/public/retail_db/orders/part-00000"
headers = ['order_id', 'order_date', 'order_customer_id', 'order_status']
orders = extract_txt(path, headers)

<class 'pyspark.rdd.PipelinedRDD'>


In [132]:
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)



In [133]:
orders.show(5)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
+--------+--------------------+-----------------+---------------+
only showing top 5 rows



In [ ]:
spark.sql("DESCRIBE TABLE customers")

In [139]:
spark.catalog.setCurrentDatabase('band_retail')

In [141]:
customers = spark.sql("SELECT * FROM customers")

In [142]:
customers.printSchema()

root
 |-- customer_city: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)



In [155]:
customers = spark.sql("SELECT * FROM customers")

In [ ]:
customers.show()

In [168]:
filter_not_order = orders.join(customers, on=customers.customer_id != orders.order_customer_id).sort('customer_lname', 'customer_fname')

In [177]:
counting = crimes.groupby('Primary Type').count().sort(desc('count'))

In [178]:
counting.show(5)

+---------------+-------+
|   Primary Type|  count|
+---------------+-------+
|          THEFT|1570224|
|        BATTERY|1368734|
|CRIMINAL DAMAGE| 850873|
|      NARCOTICS| 740084|
|        ASSAULT| 479748|
+---------------+-------+
only showing top 5 rows



In [173]:
counting.printSchema()

root
 |-- Primary Type: string (nullable = true)
 |-- count: long (nullable = false)



In [182]:
#--------------------ex4---------------------------------------#

In [15]:
path = "/user/exercise/nyse_parquet"
orders.write.parquet(path)

In [16]:
#--------------------ex5---------------------------------------#

In [35]:
path = "/public/randomtextwriter"
count_text = spark.sparkContext.textFile(path)

In [36]:
count_text.collect()

['xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'xin chao',
 'toi rat vui duoc gap cac ban',
 'cac ban co khoe khong',
 'vui lam',
 'vui lam',
 'vui lam',
 'vui lam',
 'vui lam',
 'vui lam',
 'vui lam',
 'vui vui vui vui ghe vui ghe cac ban a']

In [43]:
def splitRow(textRow, saveText, listCount):
    l = textRow.split(' ')
    for text in l:
        if text not in saveText:
            saveText.append(text)
            listCount.append(1)
        elif text in saveText:
            listCount[saveText.index(text)]+=1

In [44]:
def countText(listText):
    saveText, listCount = [], []
    for textRow in listText:
        splitRow(textRow, saveText, listCount)
    return saveText, listCount

In [45]:
a,b = countText(count_text.collect())

In [90]:
print(a)
print(b)
data = list(map(lambda w,c: f'{w},{c}', a, b))
print(data)

['xin', 'chao', 'toi', 'rat', 'vui', 'duoc', 'gap', 'cac', 'ban', 'co', 'khoe', 'khong', 'lam', 'ghe', 'a']
[17, 17, 1, 1, 13, 1, 1, 3, 3, 1, 1, 1, 7, 2, 1]
['xin,17', 'chao,17', 'toi,1', 'rat,1', 'vui,13', 'duoc,1', 'gap,1', 'cac,3', 'ban,3', 'co,1', 'khoe,1', 'khong,1', 'lam,7', 'ghe,2', 'a,1']


In [95]:
rdd = spark.sparkContext.parallelize(data)
row_rdd = rdd.map(lambda x: Row(x))
#df=sqlContext.createDataFrame(row_rdd,['numbers']).show()
#schem = StructType().add('word,count', StringType(), True)
df = spark.createDataFrame(data=row_rdd, schema=['word,count'])

In [96]:
df.show()

+----------+
|word,count|
+----------+
|    xin,17|
|   chao,17|
|     toi,1|
|     rat,1|
|    vui,13|
|    duoc,1|
|     gap,1|
|     cac,3|
|     ban,3|
|      co,1|
|    khoe,1|
|   khong,1|
|     lam,7|
|     ghe,2|
|       a,1|
+----------+



In [ ]:
df.write.format('Avro').save('/user/exercise/solutions/solution05/wordcount')

In [100]:
spark.catalog.setCurrentDatabase('band_retail')

2022-01-04 16:41:51,212 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [104]:
spark.catalog.listTables()

[Table(name='customers', database='band_retail', description=None, tableType='MANAGED', isTemporary=False)]

In [105]:
spark.sql("SELECT * FROM customers").show()

+-------------+--------------+--------------+-----------+--------------+--------------------+--------------+---------------+----------------+
|customer_city|customer_email|customer_fname|customer_id|customer_lname|   customer_password|customer_state|customer_street|customer_zipcode|
+-------------+--------------+--------------+-----------+--------------+--------------------+--------------+---------------+----------------+
|            1|       Richard|     Hernandez|       null|     XXXXXXXXX|  6303 Heather Plaza|   Brownsville|             TX|           78521|
|            2|          Mary|       Barrett|       null|     XXXXXXXXX|9526 Noble Embers...|     Littleton|             CO|           80126|
|            3|           Ann|         Smith|       null|     XXXXXXXXX|3422 Blue Pioneer...|        Caguas|             PR|           00725|
|            4|          Mary|         Jones|       null|     XXXXXXXXX|  8324 Little Common|    San Marcos|             CA|           92069|
|     